In [ ]:
# commands to ge stockfish resources
!pip install python-chess==0.31.0
!pip install pip install chess-board
!wget https://www.dropbox.com/sh/75gzfgu7qo94pvh/AACk_w5M94GTwwhSItCqsemoa/Stockfish%205/stockfish-5-linux.zip
!unzip stockfish-5-linux.zip
!chmod +x stockfish-5-linux/Linux/stockfish_14053109_x64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!7z e "/content/drive/MyDrive/dataset.7z"



In [ ]:
# libraries that will be needed in our program
import chess
import chess.engine
import random
import numpy
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.callbacks as callbacks
from google.colab import drive

In [ ]:
# Functions to convert a state of board into respective matrixes
index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}

def index_generator(square):
  # Takes a square from the board and converts it to a matrix displaying where on the board it wll be
  data = chess.square_name(square)
  return 8 - int(data[1]), index[data[0]]
 

def split_dimensions(board):
 # Creates a 14, 8, 8 matrix of type int8 used to store the position of each piece on the board
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # For all the types of chess pieces
  for piece in chess.PIECE_TYPES:
    # Nested loop which deals with the white pieces
    for square in board.pieces(piece, chess.WHITE):
      # For the given piece it will find the position of piece and place it in the board3d matrix which hold all the pieces positions.
      idx = numpy.unravel_index(square, (8, 8))
      # As this is a nested loop it will run through multiple times. 
      #The number of pieces will decrease resulting in a list of all white pieces positions displayed on a 14x8x8 matrix 
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      # The same will occur for the black pieces.
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  # creating a variable for whether it is white pieces turn or black pieces turn
  #for the white moves
  board.turn = chess.WHITE
  #Checking all of whites legal moves and adding them to the 14x8x8 matrix
  for move in board.legal_moves:
      i, j = index_generator(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  #Does the same for the black moves also adding them to the matrix
  for move in board.legal_moves:
      i, j = index_generator(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [ ]:
board = chess.Board()
print(board)
split_dimensions(board)

In [ ]:
# CNN model  with 5 Convolutional Layers and 4 Fully Connected Layers
def build_model():
  model = layers.Input(shape=(14, 8, 8))
  # Creating a model with the input layers shape being 14x8x8

  # adding the convolutional layers
  x = model
  conv_size = 32
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  # The layers creates a convolution kernal that takes in the input layer.
  # Filters determine the size of the output filters in the convolution
  # Kernal size specifies the height and width of the convolution windown
  # Padding relates to the output being the same size and the input
  # Activation function adds non linear properties to the neural network
  # Data_format channels_first takes the input as a shape
  x = layers.BatchNormalization()(x)
  #BatchNormalization applies a transformation which maintains the mean output to 0 and the standard deviation to 1
  #Adding multiple convolutional layers
  # adding Fully Connected Layers with Relu Activation function
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters=conv_size, kernel_size=3, padding='same', activation='relu', data_format='channels_first')(x)
  x = layers.Flatten()(x)
  # Flattens the input
  
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)
 

  return models.Model(inputs=model, outputs=x)

In [ ]:
model = build_model()
#calling the function to build the model
model.compile(optimizer=optimizers.Adam(1e-4), loss='mean_squared_error')
model.summary()


Skip connections (residual network) will likely improve the model for deeper connections. If you want to test the residual model, check the code below.

In [ ]:
# Method to get process the data from google drive
def get_data():
  data = numpy.load('/content/drive/MyDrive/chessdata/dataset.npz')
  #data = numpy.load('/content/dataset.npz')
  # Splitting the data into the  matrices and the value of whether black or white won the game.
  matrix = data['b']
  values = data['v']
  print(matrix[0])
  print(values)
  # the values within the data are normalised to values between 0 and 1 
  values = numpy.asarray(values / abs(values).max() / 2 + 0.5, dtype=numpy.float32) 
  return matrix, values

# Calling the get_data function to used for training the model 
x_train, y_train = get_data()
print(x_train.shape)
print(y_train.shape)

dataview = numpy.load('/content/drive/MyDrive/chessdata/dataset.npz')



In [ ]:
# Training the model using the data

model.fit(x_train, y_train,
          # batch size is the number of samples per gradient update
          batch_size=2048,
          # An epoch is an iteration over the x and y dataset. represents the number of epochs to train the model
          epochs=1000,
          # the progress bar
          verbose = 1,
          # A fraction of the training data to be used as validation
          validation_split=0.1,
          
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=8,verbose=1),
                     callbacks.EarlyStopping(monitor='loss', patience=10, min_delta=1e-4,verbose=1)])

model.save('CNN_Model.h5')


In [ ]:
# used for the minimax algorithm
def evaluator(board):
  #calling the split_dimensions method
  board = split_dimensions(board)
  #expanding the board on the 0 axis
  board = numpy.expand_dims(board, 0)
  return model.predict(board)[0][0]

# Min Max Function with alpha beta pruning
# A recursive algofrithm that calls itself whilst running
def minimax(board, depth, alpha, beta, max_player):
  if depth == 0 or board.is_game_over():

    return evaluator(board)
  
  
  if max_player == False:
    # meval is a variable used to represent positive infinity
    meval = numpy.inf
    # for every move in the chess board that is legal
    for legal_move in board.legal_moves:
      # Push the next legal move in the list of all legal moves
      board.push(legal_move)
      # The minimax function is called again for the next legal move and saved as eval
      eval = minimax(board, depth - 1, alpha, beta, True)
      # Unmake the last move
      board.pop()
      # meval in the minimum value of meval or eval
      meval = min(meval, eval)
      # beta is the minimum value of beta or eval
      beta = min(beta, eval)
      # if the beta is less than or equal to alpha 
      if beta <= alpha:
        break
    return meval
  
  # max_player = true
  else:
    # meval is a variable used to represent negative infinity 
    meval = -numpy.inf
    # for every move in the chess board that is legal 
    for legal_move in board.legal_moves:
      # Push the next legal move in the list of all legal moves
      board.push(move)
      # The minimax function is called again for the next legal move and saved as eval
      eval = minimax(board, depth - 1, alpha, beta, False)
      # Unmake the last move
      board.pop()
      # max_eval is the maximum of max_eval and eval
      max_eval = max(max_eval, eval)
      # alpha is the maximum of alpha and eval 
      alpha = max(alpha, eval)
      #if beta is less that or equal to alpha
      if beta <= alpha:
        break
    return max_eval


# this is the actual function that gets the move from the neural network
def get_move(board, depth):
  max_move = None
  #max_eval set to negative infinity
  max_eval = -numpy.inf
  #for every move in the boards legal move
  for m in board.legal_moves:
    # push the move
    board.push(m)
    # Calling the minimax function and saving it to variable eval
    eval = minimax(board, depth - 1, -numpy.inf, numpy.inf, False)
    # pop the last move
    board.pop()
    # if the eval is greater than the max_eval
    if eval > max_eval:
      #finding the move with the maximum value and setting it to max move
      max_eval = eval
      max_move = m
  
  return max_move

In [ ]:
# Computer vs Computer
board = chess.Board()
with chess.engine.SimpleEngine.popen_uci('/content/stockfish-5-linux/Linux/stockfish_14053109_x64') as engine:
  while True:
    #move is calling the get_move function
    move = get_move(board, 1)
    board.push(move)
    # Printing the board after a move has been played
    print(f'\n{board}')
    if board.is_game_over():
      break
   
    # The chess engine Stockfish analyse the board and play its get move 
    move = engine.analyse(board, chess.engine.Limit(time=1), info=chess.engine.INFO_PV)['pv'][0]
    # Pushing the move
    board.push(move)
    # Printing the board after stockfish plays a move
    print(f'\n{board}')
    if board.is_game_over():
      break

In [ ]:
# Computer vs Human
from chessboard import display
board = chess.Board()

while board.is_game_over() == False:
 
  print(f'\n{board}')
  print(board.legal_moves)
  move = input("Select moves from upper range Enter your move:")
  board.push_san(move)
  if board.is_game_over():
      print("Human Wins")
      break
  
  move = get_move(board, 1)
  board.push(move)
  print(f'\n{board}')
  if board.is_game_over():
      print("AI wins")
      break
 
  


In [ ]:
# AI vs AI
from chessboard import display
board = chess.Board('rnbqkbnr/pppp1ppp/8/4p3/5Q2/8/PPPPPPPP/RNB1KBNR')

while board.is_game_over() == False:
  move = get_move(board, 1)
  board.push(move)
  print(f'\n{board}')
  if board.is_game_over():
      print("AI  1 Wins")
      break
  move = get_move(board, 1)
  board.push(move)
  print(f'\n{board}')
  if board.is_game_over():
      print("AI 2 wins")
      break
  

In [ ]:
# block to save and load model
